# Preliminaries

In [66]:
import copy
import itertools
from collections import defaultdict
from operator import itemgetter
import folium
import numpy as np
from folium import plugins
from folium.plugins import HeatMap
import branca.colormap
from collections import defaultdict
import folium
import pandas as pd
from html2image import Html2Image
import os

In [9]:
df = pd.read_csv(r'/Users/clothildedevillenfagne/Cours/Master_2/Memoire/moyen_data.csv', sep="\t")

/Users/clothildedevillenfagne/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (9,10,14,16,17,37,38,39,40,41,43,44,45,46,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [77]:
new_df = df[df['class']=='Aves']
new_df = new_df[['species', 'year', 'month', 'decimalLatitude', 'decimalLongitude', 'individualCount']]

### Regarder pour les position

In [98]:
new_df

,family,species,year,month,decimalLatitude,decimalLongitude,individualCount
477,Phasianidae,Phasianus colchicus,2021,4.0,51.22869,4.73774,1.0
479,Anatidae,Aix galericulata,2021,4.0,50.89379,4.76185,1.0
480,Anatidae,Alopochen aegyptiaca,2021,4.0,50.81495,4.29600,2.0
481,Anatidae,Cygnus atratus,2021,4.0,51.18572,3.27233,1.0
482,Anatidae,Alopochen aegyptiaca,2021,4.0,50.75811,5.17392,2.0
...,...,...,...,...,...,...,...
22563391,Rallidae,Fulica atra,2018,12.0,51.10804,4.17848,5.0
22563392,Accipitridae,Accipiter nisus,2018,12.0,51.24820,3.39401,1.0
22563393,Rallidae,Fulica atra,2018,12.0,50.88176,4.31492,8.0
22563394,Anatidae,Aythya fuligula,2018,12.0,50.97714,3.67660,32.0


In [260]:
def ps_for_coo(df):
    latitude = df['decimalLatitude'].tolist()
    longitude = df['decimalLongitude'].tolist()
    coordonee = list(set(tuple(zip(latitude, longitude))))
    df_final = pd.DataFrame(index=coordonee)
    year = set(df['year'].tolist())
    for ye in year:
        print('Année : ', ye)
        #ye_list = []
        data1 = df[df['year']==ye]
        latitude = data1['decimalLatitude'].tolist()
        longitude = data1['decimalLongitude'].tolist()
        coordonee = list(set(tuple(zip(latitude, longitude))))
        big_list = []
        dim = len(data1)
        #for mo in month:
            #data2 = data1[data1.month == mo].copy()
        espece = set(data1['species'].tolist())
            #print('mois :', mo)
        for esp in espece:
            if pd.isnull(esp)==False:
                lst = [0] * len(coordonee)
                data2 = data1[data1['species'] == esp].copy()
                latitude_esp = data2['decimalLatitude'].tolist()
                longitude_esp = data2['decimalLongitude'].tolist()
                coordonee_esp = set(tuple(zip(latitude_esp, longitude_esp)))
                #list_esp = []
                for coo in coordonee_esp:
                    index = coordonee.index(coo)
                    lst[index] = 1
                big_list.append(lst)

        df_prefix_coo = pd.DataFrame(big_list, columns =coordonee)
        dico_coo = df_prefix_coo.sum()
        df_coo = pd.DataFrame(dico_coo, columns=[ye])
    
        df_final = pd.concat([df_final, df_coo], axis=1)
        
    
    return df_final 

In [ ]:
res = ps_for_coo(new_df)

Année :  2000
Année :  2001
Année :  2002
Année :  2003
Année :  2004
Année :  2005
Année :  2006
Année :  2007
Année :  2008
Année :  2009
Année :  2010
Année :  2011
Année :  2012
Année :  2013
Année :  2014
Année :  2015
Année :  2016
Année :  2017
Année :  2018
Année :  2019
Année :  2020
Année :  2021


In [ ]:
df_max_scaled = res.copy() 
  
for column in df_max_scaled.columns: 
    df_max_scaled[column] = df_max_scaled[column]  / df_max_scaled[column].abs().max() 
      
display(df_max_scaled)

In [63]:
def map_coo (df, annee):
    count = df[annee].tolist()
    coo = df.index.tolist()
    #coo = test_df['coordonee'].tolist()
    #compte = test_df['compte'].tolist()
    
    loc = [4.35, 50.8333]
    mappy = folium.Map(location=[loc[1], loc[0]], zoom_start=8)
    heat = []
    for i in range(len(coo)):
        if pd.isnull(count[i])==False:
            value = int(count[i]*2)
            for j in range(value):
                heat.append(list(coo[i]))
    steps=20
    colormap = branca.colormap.linear.YlOrRd_09.scale(0, 1).to_step(steps)
    gradient_map=defaultdict(dict)
    for i in range(steps):
        gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
    colormap.add_to(mappy)
    HeatMap(heat,gradient = gradient_map).add_to(mappy)

    return mappy

In [ ]:
map(new_df,2018)

# Analyse nombre

In [ ]:
def ps_for_num(df):
    latitude = df['decimalLatitude'].tolist()
    longitude = df['decimalLongitude'].tolist()
    coordonee = list(set(tuple(zip(latitude, longitude))))
    df_final = pd.DataFrame(index=coordonee)
    year = set(df['year'].tolist())
    for ye in year:
        print('Année : ', ye)
        #ye_list = []
        data1 = df[df['year']==ye]
        latitude = data1['decimalLatitude'].tolist()
        longitude = data1['decimalLongitude'].tolist()
        coordonee = list(set(tuple(zip(latitude, longitude))))
        big_list = []
        dim = len(data1)
        #for mo in month:
            #data2 = data1[data1.month == mo].copy()
        espece = set(data1['species'].tolist())
            #print('mois :', mo)
        for esp in espece:
            if pd.isnull(esp)==False:
                lst = [0] * len(coordonee)
                data2 = data1[data1['species'] == esp].copy()
                latitude_esp = data2['decimalLatitude'].tolist()
                longitude_esp = data2['decimalLongitude'].tolist()
                coordonee_esp = set(tuple(zip(latitude_esp, longitude_esp)))
                ind_count = final_df['individualCount'].tolist()
                for i in range(len(coordonee_esp)):
                    index = coordonee.index(coordonee_esp[i])
                    lst[index] += ind_count[i]
                big_list.append(lst)

        df_prefix_coo = pd.DataFrame(big_list, columns =coordonee)
        dico_coo = df_prefix_coo.sum()
        df_coo = pd.DataFrame(dico_coo, columns=[ye])
        #dico_final={}
        #for key, value in dico_coo.items():
        #    dico_final[key]=value/dim
        #print(dico_final)

        '''loc = [4.35, 50.8333]
        mappy = folium.Map(location=[loc[1], loc[0]], zoom_start=8)
        heat = []
        for key, value in dico_final.items():
            for j in range(int(value*3000)):
                heat.append(list(key))
        steps=20
        colormap = branca.colormap.linear.YlOrRd_09.scale(0, 1).to_step(steps)
        gradient_map=defaultdict(dict)
        for i in range(steps):
            gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
        colormap.add_to(mappy)

        HeatMap(heat, gradient = gradient_map).add_to(mappy)'''
        df_final = pd.concat([df_final, df_coo], axis=1)
        #print(df_final)
    
    return df_final #resultat_final
        